In [2]:
%reload_ext autoreload
%autoreload 2

In [1]:
# Optimisation parameters
problem_name = "Forrester"
problem_params = {}
time_name = "negcorrtime"
n_workers = 1
acq_name = "LocalPenalisationBatchBO"
bo_name = "AsyncProblemDepentantBO"
run_no = 1
budget = 1000

acq_params = {
    "n_opt_samples": 1 * 1000,
    "n_opt_bfgs": 10,
    "acq_name": "EI",
}

# Left as defaults
save_every = 10
repeat_no = None

In [4]:
import aegis
import torch
import os
from aegis import test_problems, time_dists, batch, optim

# set up the saving paths
save_path = aegis.util.generate_save_filename(
    time_name,
    problem_name,
    budget,
    n_workers,
    acq_name,
    run_no,
    bo_name,
    problem_params,
    acq_params,
    repeat_no=repeat_no,
)

if os.path.exists(save_path):
    load_path = save_path
    print("Loading saved run")
else:
    load_path = aegis.util.generate_data_filename(
        problem_name, run_no, problem_params, repeat_no=repeat_no
    )

# load the training data
data = torch.load(load_path)
Xtr = data["Xtr"]
Ytr = data["Ytr"]

# if it has additional arguments add them to the dictionary passed to f
if "problem_params" in data:
    problem_params.update(data["problem_params"])

print(f"Training data shape: {Xtr.shape}")

# load the problem instance
f = getattr(test_problems, problem_name)(**problem_params)

# wrap the problem for torch and so that it resides in [0, 1]^d
f = aegis.util.TorchProblem(aegis.util.UniformProblem(f))

# instantiate the time function
time_class = getattr(time_dists, time_name)

# get the acquisition function class
acq_class = getattr(batch, acq_name)

# get the BO class
asbo = getattr(optim, bo_name)

# run the BO
asbo = asbo(
    f,
    Xtr,
    Ytr,
    acq_class,
    acq_params,
    budget,
    n_workers,
    time_class,
    q=1,
    verbose=False,
)

KeyError: 'acq_name'

In [3]:
import matplotlib.pyplot as plt
import plotting
import numpy as np
from aegis.batch import ratios

asbo.step()

res = asbo.get_results()
models = asbo.get_models()

T_data = asbo.output_transform(asbo.Ytr)
T_time = asbo.output_transform(asbo.time_taken)

plt.rcParams['figure.figsize'] = [7, 7]

fig, axs = plt.subplots(4)
color = np.array([31, 119, 180])/255
plotting.gp_plot(axs[0], models["ProblemModel"], T_data, color=color)
axs[0].set(title="Problem Model")
if "CostModel" in models:
    plotting.gp_plot(axs[1], models["CostModel"], T_time, color=color)
    axs[1].set(title="Cost Model")

# Get EI for comparison
acq_params = {
    "n_opt_samples": 1 * 1000,
    "n_opt_bfgs": 10,
}
EI = ratios.EI(
    models["ProblemModel"],
    asbo.f.lb,
    asbo.f.ub,
    under_evaluation=asbo.ue.get(),
    **acq_params,
)
plotting.acq_plot(axs[2], EI)
axs[2].set(title="EI For Comparison")

asbo._update_acq()

# Acq function we're using
plotting.acq_plot(axs[3], asbo.acq)
axs[3].set(title="Acqisition Function")

# Mark our choice
plotting.mark_acq_choice_plot(axs, asbo.acq, color='red')

# Mark ongoings
for x in asbo.ue.get():
    plotting.plot_vline_axes(axs, x, color="blue")

print(asbo.ue.get())

# Mark EI's choice for comparison
plotting.mark_acq_choice_plot(axs, EI, color='green')

fig.tight_layout()

/home/lewis/anaconda3/envs/mp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'asbo' is not defined